In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils import data
from torchvision import datasets, transforms

import helper

In [2]:
# define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

# download and load training data
trainset = datasets.FashionMNIST('../input/', download=True, train=True, transform=transform)
trainloader = data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('../input/', download=True, train=False, transform=transform)
testloader = data.DataLoader(testset, batch_size=64, shuffle=True)

In [3]:
# create the network
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 10)
        
    def forward(self, x):
        # let's flatten the data here
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.output(x), dim=1)
        
        return x

In [4]:
model = Network()

images, labels = next(iter(testloader))

# get class probabilities
ps = torch.exp(model(images))
print(ps.shape)

torch.Size([64, 10])


In [5]:
top_p, top_class = ps.topk(1, dim=1)
print(top_class[:10, :])
# print(top_p[:10, :])

tensor([[4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4],
        [4]])


In [6]:
equals = top_class == labels.view(*top_class.shape)

In [7]:
equals.sum().item() * 100 / 64

12.5

In [8]:
# for i, j in testloader:
#     print(i.shape, j.shape, len(i), len(j))

In [9]:
accuracy = torch.mean(equals.type(torch.FloatTensor))

In [10]:
print(f'Accuracy = {accuracy.item() * 100}%')

Accuracy = 12.5%


In [11]:
model = Network()
criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.003)
optimizer = optim.SGD(model.parameters(), lr=0.03)

epochs = 30
steps = 0

train_losses, test_losses = [], []

for e in range(epochs):
    tot_train_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        tot_train_loss += loss.item()
        # train_losses.append(tot_train_loss)
    else:
        tot_test_loss = 0
        test_correct = 0
        
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                loss = criterion(log_ps, labels)
                tot_test_loss += loss.item()
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                test_correct += equals.sum().item()
                
        # get mean loss to enable comparison between train and test sets
        train_loss = tot_train_loss / len(trainloader)
        test_loss = tot_test_loss / len(testloader)
        
        # at completion of epoch
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
        print(
            'Epoch: {}/{}.. '.format(e+1, epochs),
            'Training Loss: {:.3f}.. '.format(train_loss),
            'Test Loss: {:.3f}.. '.format(test_loss),
            'Test Accuracy: {:.3f}'.format(test_correct / len(testloader.dataset))
        )

Epoch: 1/30..  Training Loss: 0.887..  Test Loss: 0.544..  Test Accuracy: 0.799
Epoch: 2/30..  Training Loss: 0.479..  Test Loss: 0.487..  Test Accuracy: 0.823
Epoch: 3/30..  Training Loss: 0.419..  Test Loss: 0.438..  Test Accuracy: 0.835
Epoch: 4/30..  Training Loss: 0.382..  Test Loss: 0.485..  Test Accuracy: 0.826
Epoch: 5/30..  Training Loss: 0.358..  Test Loss: 0.415..  Test Accuracy: 0.847
Epoch: 6/30..  Training Loss: 0.338..  Test Loss: 0.378..  Test Accuracy: 0.865
Epoch: 7/30..  Training Loss: 0.322..  Test Loss: 0.363..  Test Accuracy: 0.870
Epoch: 8/30..  Training Loss: 0.310..  Test Loss: 0.383..  Test Accuracy: 0.865
Epoch: 9/30..  Training Loss: 0.299..  Test Loss: 0.467..  Test Accuracy: 0.836
Epoch: 10/30..  Training Loss: 0.287..  Test Loss: 0.351..  Test Accuracy: 0.875
Epoch: 11/30..  Training Loss: 0.276..  Test Loss: 0.351..  Test Accuracy: 0.876
Epoch: 12/30..  Training Loss: 0.267..  Test Loss: 0.399..  Test Accuracy: 0.857
Epoch: 13/30..  Training Loss: 0.260.